https://colab.research.google.com/drive/1uFJBO1pgsiFwCGIJwZlhUzaJ2srDbtw-#scrollTo=Fesymsn34v7z

# Import libraries

In [ ]:
import torchvision
import torch 
import torch.nn as nn 
import torch.functional as F 
import torch.optim as optim
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

In [ ]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import numpy as np 
import unicodedata
import re 
import time 
import os 

In [ ]:
torch.__version__

'1.9.0+cu111'

In [ ]:
cd drive/MyDrive/Colab\ Notebooks/Daily_Coding/Seq2seq+attention

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/Daily_Coding/Seq2seq+attention'
/content/drive/MyDrive/Colab Notebooks/Daily_Coding/Seq2seq+attention


In [ ]:
ls

 data/  'machine translation.ipynb'   Seq2seq+attention.ipynb


In [ ]:
ls data/train-en-vi

train.en  train.vi


# Prepare dataset

In [ ]:
DATA_PATH = "./data/"
TRAIN_SRC_PATH = os.path.join(DATA_PATH, "train-en-vi/train.en")
TRAIN_TGT_PATH = os.path.join(DATA_PATH, "train-en-vi/train.vi")
DEV_SRC_PATH = os.path.join(DATA_PATH, "dev-en-vi/tst2012.en")
DEV_TGT_PATH = os.path.join(DATA_PATH, "dev-en-vi/tst2012.vi")
TEST_SRC_PATH = os.path.join(DATA_PATH, "test-en-vi/tst2013.en")
TEST_TGT_PATH = os.path.join(DATA_PATH, "test-en-vi/tst2013.vi")

In [ ]:
train_src_sentences = open(TRAIN_SRC_PATH, "r").read().split("\n")
train_tgt_sentences = open(TRAIN_TGT_PATH, "r").read().split("\n")
dev_src_sentences = open(DEV_SRC_PATH, "r").read().split("\n")
dev_tgt_sentences = open(DEV_TGT_PATH, "r").read().split("\n")
test_src_sentences = open(TEST_SRC_PATH, "r").read().split("\n")
test_tgt_sentences = open(TEST_TGT_PATH, "r").read().split("\n")


In [ ]:
train_sentences = pd.DataFrame(train_src_sentences, columns=["eng"])
train_sentences["vi"] = train_tgt_sentences
train_sentences.head(5)

,eng,vi
0,Rachel Pike : The science behind a climate hea...,Khoa học đằng sau một tiêu đề về khí hậu
1,"In 4 minutes , atmospheric chemist Rachel Pike...","Trong 4 phút , chuyên gia hoá học khí quyển Ra..."
2,I &apos;d like to talk to you today about the ...,Tôi muốn cho các bạn biết về sự to lớn của nhữ...
3,Headlines that look like this when they have t...,Có những dòng trông như thế này khi bàn về biế...
4,They are both two branches of the same field o...,Cả hai đều là một nhánh của cùng một lĩnh vực ...


In [ ]:
dev_sentences = pd.DataFrame(dev_src_sentences, columns=["eng"])
dev_sentences["vi"] = dev_tgt_sentences
dev_sentences.head(5)

,eng,vi
0,How can I speak in 10 minutes about the bonds ...,Làm sao tôi có thể trình bày trong 10 phút về ...
1,This is not a finished story .,Câu chuyện này chưa kết thúc .
2,It is a jigsaw puzzle still being put together .,Nó là một trò chơi ghép hình vẫn đang được xếp .
3,Let me tell you about some of the pieces .,Hãy để tôi kể cho các bạn về vài mảnh ghép nhé .
4,Imagine the first piece : a man burning his li...,Hãy tưởng tượng mảnh đầu tiên : một người đàn ...


In [ ]:
test_sentences = pd.DataFrame(test_src_sentences, columns=["eng"])
test_sentences["vi"] = test_tgt_sentences
test_sentences.head(5)

,eng,vi
0,"When I was little , I thought my country was t...","Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên ..."
1,And I was very proud .,Tôi đã rất tự hào về đất nước tôi .
2,"In school , we spent a lot of time studying th...","Ở trường , chúng tôi dành rất nhiều thời gian ..."
3,Although I often wondered about the outside wo...,Mặc dù tôi đã từng tự hỏi không biết thế giới ...
4,"When I was seven years old , I saw my first pu...","Khi tôi lên 7 , tôi chứng kiến cảnh người ta x..."


In [ ]:
def preprocess_sentence(w):
    return '<start> ' + w + ' <end>'

In [ ]:
train_sentences["eng"] = train_sentences["eng"].apply(lambda w: preprocess_sentence(w))
train_sentences["vi"] = train_sentences["vi"].apply(lambda w: preprocess_sentence(w))
train_sentences.sample(10)


,eng,vi
75633,"<start> Now in a sense , when we think about t...","<start> Nói cách khác , khi chúng ta nghĩ về ả..."
12887,<start> So we have to keep an eye on that . <end>,"<start> Vì vậy , chúng tôi phải để mắt đến điề..."
81207,"<start> For example , these were state-of-the-...","<start> Ví dụ , đây là những hệ thống vũ khí t..."
96218,<start> You put them in a portfolio and you tr...,<start> Bạn đặt chúng vào một cặp hồ sơ và cố ...
33516,<start> And the conclusion was it was tasks th...,<start> Và kết luận là những yêu cầu có đe doạ...
98182,"<start> Now , if I came up here , and I wish I...",<start> Nếu tôi được đứng ở đây và tôi ước rằn...
106471,"<start> If we look at biology , and many of yo...","<start> Nếu chúng ta nhìn vào sinh học , và nh..."
12341,"<start> So , completely irrational , you would...","<start> Vì thế , hoàn toàn vô lý bạn sẽ nghĩ v..."
41331,<start> So this was interesting . <end>,<start> Nên điều này rất thú vị . <end>
46387,"<start> But as soon as the wolves arrived , ev...","<start> Nhưng ngay sau khi những con sói đến ,..."


#Building Vocabulary Index

class sau dùng để tạo  ra vocabulary index mapping input, output thành index trong câu:

In [ ]:
# this class create word--> index (eg: "dad":5)
# and index--> word (5-->"dad") for each language

class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang 
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            #update with individual (rieng bieet) tokens
            self.vocab.update(phrase.split(' '))

        #sort vocab
        self.vocab = sorted(self.vocab)

        #add a padding token with index 0
        self.word2idx['<pad>'] = 0
        
        #word to index mapping
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1 #plus 1 because add pad token

        #index to word mapping:
        for word, index in self.word2idx.items():
            self.idx2word[index] = word


In [ ]:
input_lang = LanguageIndex(train_sentences["eng"].values.tolist())
target_lang = LanguageIndex(train_sentences["vi"].values.tolist())

In [ ]:
input_tensor = [[input_lang.word2idx[token] for token in sent.split(' ')] for sent in train_sentences["eng"].values.tolist()]
target_tensor = [[target_lang.word2idx[token] for token in sent.split(' ')] for sent in train_sentences["vi"].values.tolist()]

In [ ]:
input_tensor[0]

[1609, 13405, 12761, 1606, 16100, 45808, 20468, 17799, 23083, 32178, 1608]

In [ ]:
target_tensor[0]

[1714,
 8047,
 18633,
 25276,
 22178,
 20214,
 23141,
 25301,
 24219,
 18965,
 18596,
 1713]

In [ ]:
def max_length(sents):
    return max(len(t) for t in sents)

In [ ]:
count = 0
for i,t in enumerate(input_tensor):
    if len(t) > 300:
        count+=1

In [ ]:
print(count)

15


In [ ]:
max_input_len, max_output_len = max_length(input_tensor), max_length(target_tensor)
print(f'max length of input and target: {max_input_len} and {max_output_len}')

max length of input and target: 630 and 852


Ta thấy nếu để max_len = 630 hay 852 thì nó quá dài để traing, chỉ 1 lượng nhỏ data có len > 300 nên ta sẽ để default MAX_LEN = 300

In [ ]:
# Max length dài quá nên chắc phải để 300 tính cho nhanh, vì hầu hết sentences có len <300
MAX_LEN = 300

In [ ]:
def pad_sequences(x, max_len):
    padded = np.zeros((max_len), dtype = np.int64)
    if len(x) > max_len:
        padded[:] = x[:max_len]
    else: 
        padded[:len(x)] = x

    return padded

In [ ]:
input_tensor = [pad_sequences(x, MAX_LEN) for x in input_tensor]
target_tensor = [pad_sequences(x, MAX_LEN) for x in target_tensor]

In [ ]:
torch.tensor(input_tensor).shape

torch.Size([133318, 300])

In [ ]:
torch.tensor(target_tensor).shape

torch.Size([133318, 300])

In [ ]:
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor,test_size = 0.2)

print(len(input_tensor_train), len(input_tensor_val), len(target_tensor_train), len(target_tensor_val))

106654 26664 106654 26664


# Load data into DataLoader for Batching



In [ ]:
from torch.utils.data import Dataset, DataLoader

In [ ]:
# convert data to tensors, pass to DadaLoader 

class MyData(Dataset):
    def __init__(self, X,y):
        self.data = X 
        self.target = y
        
        #length of sentence = len(tensor) - len(pad)
        self.length = [np.sum(1 - np.equal(x, 0)) for x in X]

    def __getitem__(self,index):
        x = self.data[index]
        y = self.target[index]
        x_len = self.length[index]
        return x,y,x_len

    def __len__(self):
        return len(self.data)

In [ ]:
train_dataset = MyData(input_tensor_train, target_tensor_train)


In [ ]:
train_dataset.data[0]

# Parameters 


In [ ]:
BUFFER_SIZE = len(input_tensor_train)
BATCH_SIZE = 32 
N_BATCH = BUFFER_SIZE//BATCH_SIZE
embedding_dim = 256
units = 1024 
vocab_input_size = len(input_lang.word2idx)
vocab_target_size = len(target_lang.word2idx)

train_dataset = MyData(input_tensor_train, target_tensor_train)
val_dataset = MyData(input_tensor_val, target_tensor_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)


# Encoder 

`pack` sequences trong Pytorch là gì?

* khi training RNN(LSTM or GRU) thì độ giàu của các câu trong batch khác nhau. 
Ví dụ: nếu sequences với batch = 8 có size lần lượt là: `[4,6,8,5,4,3,7,8]` thì bạn sẽ phải padding tất các các câu để có độ dài = max_sent = 8. Khi đó cần 64 phép tính (8x8) nhưng trên thực tế thì chỉ cần 45 là đủ. Tuy nhiên nếu muốn sử dụng fancy RNN (e.g. biLSTM) thì nó sẽ trở nên tính toán hard hơn khi padding và sẽ thực hiện nhiều phép tính hơn mức cần thiết. 

* Vì thế Pytorch cho phép pack sequence, trong packed sequence 2 có tuple. 1 tuple chứ các elements của sequence. tuple còn lại chứa size of each sequence trong batch.

Ref: https://stackoverflow.com/questions/51030782/why-do-we-pack-the-sequences-in-pytorch

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_size):
        super(Encoder, self).__init__()
        self.batch_size = batch_size
        self.enc_units = enc_units 
        self.vocab_size = vocab_size 
        self.embedding_dim = embedding_dim 
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim, self.enc_units)

    def forward(self,x, lens, device):

        #x: max_length x batch 
        x = self.embedding(x)
        # max_length x batch x embedding_dim
        x = pack_padded_sequence(x, lens) #unpad 

        self.hidden = self.initialize_hidden_state(device)
        # 1 x batch x embedding_dim

        output, self.hidden = self.gru(x, self.hidden)
        # output.shape: max_length x batch x hidden_dim
        #self.hidden: 1 x batch x
        output, _ = pad_packed_sequence(output)
        # print(output.shape)

        return output, self.hidden

    def initialize_hidden_state(self, device):
        return torch.zeros((1,self.batch_size, self.enc_units)).to(device)

In [ ]:
### sort batch function to be able to use with pad_packed_sequence
def sort_batch(X, y, lengths):
    lengths, indx = lengths.sort(dim=0, descending=True)
    X = X[indx]
    y = y[indx]
    return X.transpose(0,1), y, lengths # transpose (batch x seq) to (seq x batch)

## Test Encoder

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

test_encoder = Encoder(vocab_input_size, embedding_dim, units, BATCH_SIZE).to(device)

it = iter(train_loader)
x,y,x_len = next(it)
x,y,x_len = sort_batch(x,y,x_len)

out_enc, hidden_enc = test_encoder(x.to(device), x_len, device)

torch.Size([56, 32, 1024])


In [ ]:
out_enc.shape


torch.Size([56, 32, 1024])

In [ ]:
hidden_enc.shape

torch.Size([1, 32, 1024])

In [ ]:
x_len

tensor([56, 42, 37, 37, 36, 35, 35, 33, 33, 31, 30, 27, 25, 25, 25, 24, 22, 22,
        19, 19, 15, 12, 12, 11, 11, 11, 11, 10,  9,  8,  5,  5])

# Decoder

<img src = "https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg">

<img src = "https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg">

Step:
* score = FC(tanh(FC(EO) + FC(H)))
* attention_weights = softmax(score, axis = 1) #batch x max_length x 1
* context_vector = sum(attention_weight x output_enc, axis = 1)
* embedding_output = input decoder pass qua embedding layer
* merged vector = concat(embedding_output, context_vector) 




In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, embedding_dim, dec_units, enc_units, batch_size):
        super(Decoder, self).__init__()
        self.batch_size = batch_size
        self.dec_units = dec_units
        self.enc_units = enc_units
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.embedding = nn.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = nn.GRU(self.embedding_dim + self.enc_units, self.dec_units, batch_first= True)
        self.fc = nn.Linear(self.enc_units, self.vocab_size)

        #use for attention:
        self.W1 = nn.Linear(self.enc_units, self.dec_units)
        self.W2 = nn.Linear(self.enc_units, self.dec_units)
        self.V = nn.Linear(self.enc_units, 1)

    def forward(self, x, hidden, enc_output):

        #enc_out: max_len x batch x hidden_dim 
        enc_output = enc_output.permute(1,0,2)
        #enc_out: batch x max_len x hidden_dim 

        #hidden : batch x hidden_dim --> permute: batch x 1 x hidden_dim 
        hidden = hidden.permute(1,0,2)

        #use bahanau equal we hat:
        score = self.V(torch.tanh(self.W1(enc_output) + self.W2(hidden)))
        # batch x max_len x 1

        attention_weights = torch.softmax(score, dim = 1) 
        #batch x max_len x 1 

        #context vector:
        context_vector = torch.sum(attention_weights * enc_output, dim = 1)
        # (batch x max_len x 1) * (batch x max_len x hidden_dim) = batch x max_len x hidden_dim 
        # sum--> batch x hidden_dim
        # print(f'shape of context vector is {context_vector.shape}')
        x = self.embedding(x)
        #batch x 1 x embedding_dim 
        x = torch.cat((context_vector.unsqueeze(1), x), -1)
        # x = batch x 1 x (enc_dim + dec_dim)

        #pass qua GRU:
        output, state = self.gru(x)
        #output: batch x 1 x hidden_dim 
        #state: batch x 1 x hidden_dim 

        output = output.view(-1, output.size(2))
        #batch x hidden_dim

        x = self.fc(output)

        return x, state, attention_weights

## Test Decoder

In [ ]:
test_decoder = Decoder(vocab_target_size, embedding_dim, units, units, BATCH_SIZE).to(device)

In [ ]:
hidden_dec = hidden_enc 


In [ ]:
input_dec = torch.tensor([[target_lang.word2idx['<start>']]] * BATCH_SIZE)

In [ ]:
input_dec.shape

torch.Size([32, 1])

In [ ]:
for t in range(1, y.size(1)):
    x,state, attn_weights = test_decoder(input_dec.to(device), 
                                         hidden_dec.to(device), 
                                         out_enc.to(device))

In [ ]:
hidden_dec.permute(1,0,2).shape

torch.Size([32, 1, 1024])

In [ ]:
x.shape

torch.Size([32, 25619])

In [ ]:
state.shape

torch.Size([1, 32, 1024])

In [ ]:
attn_weights.shape

torch.Size([32, 56, 1])

# Define loss, hyperparameters...

In [ ]:
criterion = nn.CrossEntropyLoss()

def loss_function(real, pred):
    """chỉ cần xem xét những giá trị khác o, những giá trị = 0 có thể mask đi"""
    # mask = real.ge(1).type(torch.cuda.FloatTensor)
    mask = 1 - np.equal(real, 0)
    loss = criterion(pred, real) * mask
    return torch.mean(loss)

In [ ]:
mask = 1 - np.equal(y, 0)

In [ ]:
mask[30]

In [ ]:
y[30]

In [ ]:
encoder = Encoder(vocab_input_size, embedding_dim, units, BATCH_SIZE)
decoder = Decoder(vocab_target_size, embedding_dim, units, units, BATCH_SIZE)

encoder.to(device)
decoder.to(device)

optimizer = optim.Adam(list(encoder.parameters())+ list(decoder.parameters()), lr = 0.001)

# Training

In [ ]:
from tqdm import tqdm

In [ ]:
EPOCHS = 10

for epoch in tqdm(range(EPOCHS)):
    start = time.time()

    encoder.train()
    decoder.train()

    total_loss = 0
    
    for (batch_idx, (input, target, input_len)) in enumerate(train_loader):
        loss = 0
        x,y,x_len = sort_batch(input,target,input_len)

        enc_output, enc_hidden = encoder(x.to(device), x_len, device)
        dec_hidden = enc_hidden

        dec_input = torch.tensor([[target_lang.word2idx['<start>']]] * BATCH_SIZE)

        for t in range(1, y.size(1)):
            pred, dec_hidden, _ = decoder(dec_input.to(device), 
                                          dec_hidden.to(device),
                                          enc_output.to(device))
            
            loss+= loss_function(y[:,t].to('cpu'), pred.to('cpu'))
            dec_input = y[:,t].unsqueeze(1)

        batch_loss = (loss/int(y.size(1)))
        total_loss += batch_loss

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        if batch_idx%100 == 0:
            print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch_idx, batch_loss.detach().item()))
    
    #TODO: save checkpoint for model 
    print('EPOCH {} LOSS {:.4f}'.format(epoch+1, total_loss/N_BATCH))
    print('Tine taken for 1 epoch {} sec\n'.format(time.time() - start))